In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import copy


In [222]:
data = pd.read_csv("../data/txn_by_dept.csv")
data.head()

,POS Txn,Dept,ID,Sales U
0,16120100160021008773,0261:HOSIERY,250,2
1,16120100160021008773,0634:VITAMINS & HLTH AIDS,102,1
2,16120100160021008773,0879:PET SUPPLIES,158,2
3,16120100160021008773,0973:CANDY,175,2
4,16120100160021008773,0982:SPIRITS,176,1


In [3]:
depts = set(data['Dept'])
transactions = set(data['POS Txn'])
ids = set(data["ID"])

In [6]:
adj_matrix = pd.DataFrame(columns = ids, index = transactions, data = np.zeros((len(transactions), len(ids))))
for row in data.iterrows():
    adj_matrix[row[1][2]].loc[row[1][0]] += 1


# mask = adj_matrix>=1
# adj_matrix_binary = adj_matrix.mask(adj_matrix >= 1, 1)
adj_matrix.head()   

,1,2,3,5,6,7,8,9,10,11,...,295,296,297,305,306,307,308,309,310,311
16120100160034005060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16120100160034005061,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16120100160034005062,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16120100160034005063,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16120100160034005064,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
support = []
relative_support = []
for col in adj_matrix.columns: 
    support.append(adj_matrix[col].sum())
    relative_support.append(adj_matrix[col].sum()/len(adj_matrix))


In [90]:
def get_candidates(k, frequents):
    j = k-1
    candidates = []    
    if k > 1:
#             print('here ', k)
            for f1 in range(len(frequents)):
                for f2 in range(f1+1, len(frequents)):
                    if frequents[f1][0:j]==frequents[f2][0:j]:
#                         print('f1', frequents[f1][0:j+1])
#                         print('f2', frequents[f2][j:])
                        candidates.append(frequents[f1][0:j+1]+frequents[f2][j:])
#             print('candidates:', ca    # just work w    # just work way through by removing one each timeay through by removing one each timendidates)
    else:
        for f1 in range(len(frequents)):
            for f2 in range(f1+1, len(frequents)):
                candidates.append([frequents[f1],frequents[f2]])
    
    return candidates

In [121]:
def frequent_item_sets(adj_matrix, C1_itemset, min_sup):
    support_dict = {}
    binary_matrix = adj_matrix.copy()
    max_itemset = len(C1_itemset)
    candidates = C1_itemset
    
    frequents = []
    supports = []
    for candidate in candidates:
        support = binary_matrix[candidate].sum()
        

        if support >= min_sup:
                frequents.append(candidate)
                supports.append(support)
#                 print(candidate, support)

    if len(frequents) == 0:
        return support_dict

    support_dict[1] = [frequents, supports]
    candidates = get_candidates(1, frequents)
                    
    for k in range(2, max_itemset):
        print(k)
        frequents = []
        supports = []
        for candidate in candidates:
            support = binary_matrix.where(binary_matrix[candidate].sum(axis=1) == len(candidate)).sum().max()
#             support = binary_matrix[candidate].sum()
            if support >= min_sup:
                frequents.append(candidate)
                supports.append(support)
#                 print(candidate, support)

                
        if len(frequents) == 0:
            print('No Frequents')
            return support_dict
        
        support_dict[k] = [frequents, supports]
#         print(frequents)
        
        
        candidates = get_candidates(k, frequents)
#         print('full candidates ', candidates)
        candidates = prune_candidates(candidates, frequents)
#         print('pruned ', candidates)

    return support_dict

In [115]:
def prune_candidates(candidates, frequent_set):
    pruned_list = []
    
    for c in range(len(candidates)):
        prune_flag = False
        for i in range(len(candidates[c])):
#             try:
            temp = copy.deepcopy(candidates[c])
            temp.pop(i)
            if temp not in frequent_set:
                prune_flag = True
                break
        if not prune_flag:
            pruned_list.append(candidates[c])
#             except:
#                 print('candidate ', candidates[c])
#                 print('temp', temp)
#                 print('pruned_list', pruned_list)
    return pruned_list

In [54]:
frequent = frequent_item_sets(adj_matrix, adj_matrix.columns, 3)

2
3
4
5
6
7
No Frequents


In [55]:
for key in frequent.keys():
    print('Item Set Length: ', key, '\n', frequent[key])

Item Set Length:  1 
 [1, 2, 3, 5, 6, 8, 9, 10, 12, 13, 14, 17, 19, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 40, 41, 42, 44, 45, 46, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 85, 86, 93, 94, 97, 99, 100, 102, 104, 105, 106, 107, 108, 109, 110, 111, 113, 122, 128, 131, 132, 133, 135, 138, 141, 143, 144, 146, 147, 148, 149, 152, 154, 155, 156, 157, 158, 160, 164, 165, 167, 169, 170, 171, 175, 176, 177, 178, 179, 182, 183, 185, 202, 204, 213, 214, 235, 250, 263, 264, 265, 266, 268, 269, 270, 272, 273, 278, 288, 292, 294, 295, 296, 297, 307, 308, 309, 310]
Item Set Length:  2 
 [[1, 26], [1, 99], [5, 26], [5, 86], [5, 99], [5, 175], [5, 176], [5, 177], [5, 270], [6, 99], [6, 175], [6, 177], [8, 9], [12, 72], [13, 175], [19, 250], [21, 175], [23, 250], [23, 294], [24, 61], [24, 93], [24, 99], [24, 100], [24, 122], [24, 138], [24, 143], [24, 146], [24, 160], [24, 175], [24, 176], [24, 177], [

In [194]:
def get_antecedents(parent_set, frequency_dict, max_key):
    antecedents = []
    for key in frequency_dict.keys():
        if key < max_key:
            f_set = frequency_dict[key]
#             print('f_set_can', f_set[0])
#             print('f_set_sup', f_set[1])
            for i in range(len(f_set[0])):
                if len(f_set[0][i])==1:
                    temp = [f_set[0][i]]
                else:
                    temp = f_set[0][i]
#                 print(set(temp))
#                 print(set(parent_set))
                if set(temp).issubset(set(parent_set)):
#                     print(temp,f_set[1][i])
                    antecedents.append([temp,f_set[1][i]])
                
        else:
            break
                                        
                    
    return antecedents
                
    

In [251]:
def get_association_rules(frequency_dict, min_conf):
    confidence_dict = []
    for key in frequency_dict.keys():
        if key > 1:
            for parent_set in frequency_dict[key][0]:
                print('parent set: ', parent_set)
                antecedents = get_antecedents(parent_set, frequency_dict, key)
                print('antecedents: ', antecedents)
                
#                 for antecedent in antecedents:
                while len(antecedents) > 0:
                    antecedent = antecedents.pop()
                    x = set(antecedent[0])
                    x_sup = antecedent[1]
                    print('X', x, x_sup)
                    y = set(parent_set)-x
                    for compliment in antecedents:
                        if set(compliment[0]) == y:
                            y_sup = compliment[1]
                            print('Y', y, y_sup)
                    confidence = y_sup/x_sup
                    print(x, '->', y, confidence)
                    if confidence > min_conf:
                        confidence_dict.append([[list(x)],[list(y)], confidence])
                    else:
                        antecedents = prune_antecedents(x, antecedents)
                        print('pruned', antecedents)
                            
    return confidence_dict

In [218]:
def prune_antecedents(x, antecedents):
    pruned_antecedents = []
    for antecedent in antecedents:
        if set(antecedent[0]).issubset(x):
            pass
        else:
            pruned_antecedents.append(antecedent)
            
    return pruned_antecedents

In [48]:
test1 = [
[1, 0, 1, 1, 0, 0],
[0, 1, 0, 1, 0, 0],
[1, 1, 1, 0, 1, 0],
[0, 1, 0, 1, 0, 1]#,
# [1, 1, 1, 1, 1, 0]
]

test2 = [
[1, 1, 1, 0, 0],
[1, 1, 1, 1, 1],
[1, 0, 1, 1, 0],
[1, 0, 1, 1, 1],
[1, 1, 1, 1, 0]
]

In [49]:
test_df = pd.DataFrame(data=test2, columns = ['A', 'B', 'C', 'D', 'E'])
test_df.head()

,A,B,C,D,E
0,1,1,1,0,0
1,1,1,1,1,1
2,1,0,1,1,0
3,1,0,1,1,1
4,1,1,1,1,0


In [139]:
frequent_test = frequent_item_sets(test_df, test_df.columns, 2)
print('\n')
for key in frequent_test.keys():
    print('Item Set Length: ', key, '\n', frequent_test[key][0])

2
3
4


Item Set Length:  1 
 ['A', 'B', 'C', 'D', 'E']
Item Set Length:  2 
 [['A', 'B'], ['A', 'C'], ['A', 'D'], ['A', 'E'], ['B', 'C'], ['B', 'D'], ['C', 'D'], ['C', 'E'], ['D', 'E']]
Item Set Length:  3 
 [['A', 'B', 'C'], ['A', 'B', 'D'], ['A', 'C', 'D'], ['A', 'C', 'E'], ['A', 'D', 'E'], ['B', 'C', 'D'], ['C', 'D', 'E']]
Item Set Length:  4 
 [['A', 'B', 'C', 'D'], ['A', 'C', 'D', 'E']]


In [252]:
confidence = get_association_rules(frequent_test, 1)

parent set:  ['A', 'B']
antecedents:  [[['A'], 5], [['B'], 3]]
X {'B'} 3
Y {'A'} 5
{'B'} -> {'A'} 1.6666666666666667
X {'A'} 5
{'A'} -> {'B'} 1.0
pruned []
parent set:  ['A', 'C']
antecedents:  [[['A'], 5], [['C'], 5]]
X {'C'} 5
Y {'A'} 5
{'C'} -> {'A'} 1.0
pruned [[['A'], 5]]
X {'A'} 5
{'A'} -> {'C'} 1.0
pruned []
parent set:  ['A', 'D']
antecedents:  [[['A'], 5], [['D'], 4]]
X {'D'} 4
Y {'A'} 5
{'D'} -> {'A'} 1.25
X {'A'} 5
{'A'} -> {'D'} 1.0
pruned []
parent set:  ['A', 'E']
antecedents:  [[['A'], 5], [['E'], 2]]
X {'E'} 2
Y {'A'} 5
{'E'} -> {'A'} 2.5
X {'A'} 5
{'A'} -> {'E'} 1.0
pruned []
parent set:  ['B', 'C']
antecedents:  [[['B'], 3], [['C'], 5]]
X {'C'} 5
Y {'B'} 3
{'C'} -> {'B'} 0.6
pruned [[['B'], 3]]
X {'B'} 3
{'B'} -> {'C'} 1.0
pruned []
parent set:  ['B', 'D']
antecedents:  [[['B'], 3], [['D'], 4]]
X {'D'} 4
Y {'B'} 3
{'D'} -> {'B'} 0.75
pruned [[['B'], 3]]
X {'B'} 3
{'B'} -> {'D'} 1.0
pruned []
parent set:  ['C', 'D']
antecedents:  [[['C'], 5], [['D'], 4]]
X {'D'} 4
Y {'

In [253]:
for c in confidence:
    print(c)

[[['B']], [['A']], 1.6666666666666667]
[[['D']], [['A']], 1.25]
[[['E']], [['A']], 2.5]
[[['D']], [['C']], 1.25]
[[['E']], [['C']], 2.5]
[[['E']], [['D']], 2.0]
[[['C', 'B']], [['A']], 1.6666666666666667]
[[['D', 'B']], [['A']], 2.5]
[[['D', 'C']], [['A']], 1.25]
[[['D', 'A']], [['C']], 1.25]
[[['C', 'E']], [['A']], 2.5]
[[['E', 'A']], [['C']], 2.5]
[[['D', 'E']], [['A']], 2.5]
[[['E', 'A']], [['D']], 2.0]
[[['D', 'B']], [['C']], 1.5]
[[['D', 'E']], [['C']], 2.5]
[[['C', 'E']], [['D']], 2.0]
[[['D', 'C', 'B']], [['A']], 2.5]
[[['D', 'A', 'B']], [['C']], 1.5]
[[['D', 'B']], [['C', 'A']], 1.5]
[[['D', 'C', 'E']], [['A']], 2.5]
[[['D', 'E', 'A']], [['C']], 2.5]
[[['C', 'E', 'A']], [['D']], 2.0]


In [248]:
def get_lifts(confidences_list, frequency_dict, size_dataset):
    for c in confidences_list:
        print(c)
        confidence = c[2]
        k = len(c[1])
        frequencies = frequency_dict[k]
        for i in range(len(frequencies[0])):
            if set(c[1])==set(frequencies[0][i]):
                support = frequencies[1][i]
        lift = confidence/(support/size_dataset)
        
#         c.append(lift)
        
        # leverage = rel_sup(XY) - rel(x)*rel(y)
        print(c[0])
        xy = set(c[0].append(c[1]))
        k = len(xy)
        frequencies = frequency_dict[k]
        for i in range(len(frequencies[0])):
            if xy==set(frequencies[0][i]):
                support_xy = frequencies[1][i]
        rel_xy = support_xy/size_dataset
        
        x = set(c[0])
        k = len(x)
        frequencies = frequency_dict[k]
        for i in range(len(frequencies[0])):
            if xy==set(frequencies[0][i]):
                support_x = frequencies[1][i]
        rel_x = support_x/size_dataset
        
        y = set(c[1])
        k = len(y)
        frequencies = frequency_dict[k]
        for i in range(len(frequencies[0])):
            if xy==set(frequencies[0][i]):
                support_y = frequencies[1][i]
        rel_y = support_y/size_dataset
        
        leverage = rel_xy - rel_x*rel_y
        
        c.append(leverage)
        

        
    return conficences_list
        


In [249]:
get_lifts(confidence, frequent_test, len(test_df))

[['B', ['A'], ['A'], ['A'], ['A'], ['A'], ['A'], ['A']], ['A'], 1.6666666666666667, 1.6666666666666667, 1.6666666666666667, 1.6666666666666667, 1.6666666666666667, 1.6666666666666667]
['B', ['A'], ['A'], ['A'], ['A'], ['A'], ['A'], ['A']]


TypeError: 'NoneType' object is not iterable